# Replicate AR6 Chapter 4 Fig 4.11

In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset, num2date

In [2]:
from src.util import RetrieveGitHub

In [3]:
owner = 'IPCC-WG1'
repo = 'Chapter-4_Figure4.11'
repo_ch4 = RetrieveGitHub(owner, repo, './datain')

## CMIP6 projections

In [4]:
scenarios = [
    'ssp119',
    'ssp126',
    'ssp245',
    'ssp370',
    'ssp585',
]

### Liang et al. (2020)

In [5]:
def wrap_read_csv(path):
    df = pd.read_csv(path)

    # mean_e: unweighted mean
    # mean_w: weighted mean
    df_mean = df.iloc[:, 3:].set_index('year')

    # cdf: weighted cdf
    # cdf_e: unweighted cdf
    df_cdf = pd.concat({
        vn: pd.DataFrame(np.vstack(d1.apply(lambda x: np.array(eval(x)))), index=df_mean.index)
        for vn, d1 in df.iloc[:, :2].iteritems()
    })

    return pd.concat([
        pd.concat({
            'Raw': df_mean['mean_e'].to_frame('Mean'),
            'Constrained': df_mean['mean_w'].to_frame('Mean'),
        }).stack().unstack('year'),

        df_cdf[[5, 50, 95]]
        .rename({'cdf': 'Constrained', 'cdf_e': 'Raw'}, level=0)
        .rename(columns=lambda x: 'Q{:02d}'.format(x))
        .stack()
        .unstack('year'),
    ]).sort_index()

In [6]:
df_liang = {}

for scenario in scenarios:
    if scenario == 'ssp119':
        continue

    scenario_sfx = scenario[3:]
    path_cdf = 'input_data/constrained_cmip6/Liang/cdf_{}.csv'.format(scenario_sfx)
    path_cdf_his = 'input_data/constrained_cmip6/Liang/cdf_his_{}.csv'.format(scenario_sfx)

    df = []

    for path in [path_cdf_his, path_cdf]:
        path_local = repo_ch4.retrieve(path)
        df.append(wrap_read_csv(path_local))

    df = pd.concat(df, axis=1)

    year = df.columns[-1] + 1
    df_liang[scenario] = pd.concat([
        df,
        df.iloc[:, -10:].apply(
            lambda d1:
            pd.Series({year: np.poly1d(np.polyfit(d1.index.values, d1.values, 1))(year)}),
            axis=1,
        ),
    ], axis=1)

[2024-07-07 17:01:33 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Liang/cdf_his_126.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Liang/cdf_his_126.csv on 2024-06-24
[2024-07-07 17:01:33 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Liang/cdf_126.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Liang/cdf_126.csv on 2024-06-24
[2024-07-07 17:01:34 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Liang/cdf_his_245.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Liang/cdf_his_245.csv on 2024-06-24
[2024-07-07 17:01:34 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Liang/cdf_245.csv retrieved from https://github.com/IPCC-WG1

In [7]:
df_liang = pd.concat(df_liang)
df_liang

1996      1997      1998      1999      2000  \
ssp126 Constrained Mean  0.026871  0.051770  0.075489  0.099032  0.123643   
                   Q05   0.009132  0.025022  0.037600  0.053013  0.065606   
                   Q50   0.023680  0.048655  0.077299  0.099533  0.126918   
                   Q95   0.042553  0.079706  0.114335  0.146747  0.178574   
       Raw         Mean  0.031298  0.058863  0.085491  0.112082  0.139065   
                   Q05   0.010618  0.023453  0.036795  0.053690  0.064892   
                   Q50   0.029930  0.054384  0.080981  0.105504  0.132784   
                   Q95   0.050302  0.095037  0.138304  0.180173  0.217973   
ssp245 Constrained Mean  0.026626  0.051581  0.075859  0.102287  0.126091   
                   Q05   0.010407  0.022997  0.039268  0.051336  0.063366   
                   Q50   0.024429  0.048632  0.068447  0.095112  0.125261   
                   Q95   0.042067  0.082515  0.118932  0.157695  0.188719   
       Raw         Mean  0.031790  0.059601  0.085901  0.113147  0.139742   
                   Q05   0.012204  0.024936  0.039181  0.050666  0.063179   
                   Q50   0.029911  0.056708  0.082218  0.110132  0.135721   
                   Q95   0.050691  0.091864  0.132294  0.171169  0.208376   
ssp370 Constrained Mean  0.026048  0.047994  0.072588  0.099661  0.122670   
                   Q05   0.008612  0.019619  0.031981  0.048493  0.063868   
                   Q50   0.023816  0.045294  0.068898  0.095748  0.118271   
                   Q95   0.043593  0.081639  0.125504  0.165599  0.203799   
       Raw         Mean  0.030038  0.056290  0.084019  0.112964  0.138278   
                   Q05   0.010473  0.022606  0.034962  0.049330  0.064032   
                   Q50   0.026906  0.051883  0.077854  0.109761  0.131390   
                   Q95   0.050210  0.092171  0.134745  0.178027  0.219963   
ssp585 Constrained Mean  0.026269  0.052238  0.076383  0.101806  0.127891   
                   Q05   0.009610  0.023571  0.040457  0.057855  0.067495   
                   Q50   0.023857  0.047953  0.073713  0.098811  0.126923   
                   Q95   0.043121  0.078997  0.115097  0.156247  0.199812   
       Raw         Mean  0.031761  0.059519  0.086386  0.114758  0.143057   
                   Q05   0.011410  0.023201  0.037340  0.054420  0.065578   
                   Q50   0.029673  0.054519  0.083433  0.111086  0.137535   
                   Q95   0.051072  0.093124  0.133734  0.173108  0.214774   

                             2001      2002      2003      2004      2005  \
ssp126 Constrained Mean  0.146332  0.168519  0.193447  0.217495  0.242564   
                   Q05   0.073845  0.084726  0.105236  0.122211  0.131038   
                   Q50   0.148448  0.170912  0.190793  0.214018  0.235476   
                   Q95   0.204082  0.236742  0.274865  0.308458  0.358036   
       Raw         Mean  0.164143  0.189112  0.216426  0.244049  0.272224   
                   Q05   0.076412  0.089096  0.107756  0.122461  0.135308   
                   Q50   0.154374  0.176022  0.196888  0.227025  0.252821   
                   Q95   0.250744  0.288683  0.327679  0.366224  0.407260   
ssp245 Constrained Mean  0.147114  0.167014  0.192339  0.217083  0.241029   
                   Q05   0.076642  0.085671  0.103829  0.111026  0.131399   
                   Q50   0.153059  0.175111  0.198179  0.223488  0.250274   
                   Q95   0.203790  0.228354  0.262329  0.297069  0.334462   
       Raw         Mean  0.164120  0.187401  0.213610  0.239961  0.266803   
                   Q05   0.078698  0.086953  0.098375  0.108539  0.130089   
                   Q50   0.160948  0.185397  0.213854  0.238281  0.261965   
                   Q95   0.244510  0.287132  0.328602  0.367802  0.413735   
ssp370 Constrained Mean  0.143553  0.163573  0.184680  0.206234  0.227528   
                   Q05   0.071151  0.078340  0.088184  0.096111  0.112348   
                   Q50  

### Tokarska et al. (2020)

In [8]:
map_names = {
    '5th percentile': 'Q05',
    '95th percentile': 'Q95',
    'mean': 'Mean',
    'median': 'Q50',
}

In [9]:
df_tokarska = {}

for scenario in scenarios:
    path = f'input_data/constrained_cmip6/Tokarska/{scenario}.csv'
    path_local = repo_ch4.retrieve(path)
    df_tokarska[(scenario, 'Constrained')] = (
        pd.read_csv(path_local).set_index('year').T.loc[list(map_names)]
    )

[2024-07-07 17:01:45 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp119.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Tokarska/ssp119.csv on 2024-06-24
[2024-07-07 17:01:45 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp126.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Tokarska/ssp126.csv on 2024-06-24
[2024-07-07 17:01:45 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp245.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Tokarska/ssp245.csv on 2024-06-24
[2024-07-07 17:01:45 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp370.csv retrieved from https://github.com/IPCC-WG1/C

In [10]:
# Treat raw data separately to avoid errors due to wrong headers
for scenario in scenarios:
    if scenario == 'ssp585':
        continue

    path = f'input_data/constrained_cmip6/Tokarska/{scenario}_raw.csv'
    path_local = repo_ch4.retrieve(path)
    df_tokarska[(scenario, 'Raw')] = (
        pd.read_csv(path_local).set_index('year').T
        .rename(str.strip).loc[list(map_names)]
    )

[2024-07-07 17:01:48 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp119_raw.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Tokarska/ssp119_raw.csv on 2024-06-24
[2024-07-07 17:01:48 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp126_raw.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Tokarska/ssp126_raw.csv on 2024-06-24
[2024-07-07 17:01:48 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp245_raw.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Tokarska/ssp245_raw.csv on 2024-06-24
[2024-07-07 17:01:48 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp370_raw.csv retrieved from h

In [11]:
scenario = 'ssp585'
path = f'input_data/constrained_cmip6/Tokarska/{scenario}_raw.csv'
path_local = repo_ch4.retrieve(path)
df = pd.read_csv(path_local)
df.columns = df.columns[1:].tolist() + ['dummy']
df_tokarska[(scenario, 'Raw')] = df.iloc[:, :-1].set_index('year').T.loc[list(map_names)]

[2024-07-07 17:01:51 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Tokarska/ssp585_raw.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Tokarska/ssp585_raw.csv on 2024-06-24


In [12]:
df_tokarska = pd.concat(df_tokarska).rename(map_names, level=2).sort_index()

In [13]:
year = df_tokarska.columns[-1] + 1
year

2091

In [14]:
df_tokarska = pd.concat([
    df_tokarska,
    df_tokarska.iloc[:, -10:].apply(
        lambda d1:
        pd.Series({year: np.poly1d(np.polyfit(d1.index.values, d1.values, 1))(year)}),
        axis=1,
    ),
], axis=1)

In [15]:
# Shift years
df_tokarska.columns = df_tokarska.columns.values - 10
df_tokarska

2000      2001      2002      2003      2004  \
ssp119 Constrained Mean  0.120859  0.146200  0.167669  0.190257  0.212507   
                   Q05   0.072760  0.091572  0.106083  0.117410  0.128433   
                   Q50   0.121024  0.146362  0.167864  0.190548  0.212832   
                   Q95   0.168366  0.200167  0.228505  0.262247  0.295474   
       Raw         Mean  0.138791  0.166178  0.190479  0.215593  0.240531   
                   Q05   0.082353  0.106554  0.127432  0.146123  0.164406   
                   Q50   0.134769  0.153581  0.169318  0.199241  0.224137   
                   Q95   0.224651  0.266334  0.305290  0.343688  0.380945   
ssp126 Constrained Mean  0.108532  0.131101  0.152809  0.175000  0.197138   
                   Q05   0.054799  0.066196  0.079864  0.094780  0.107520   
                   Q50   0.108693  0.131299  0.152971  0.175231  0.197373   
                   Q95   0.161623  0.195311  0.225304  0.254443  0.286030   
       Raw         Mean  0.146373  0.174011  0.200248  0.226663  0.253417   
                   Q05   0.082175  0.100805  0.113941  0.134594  0.152175   
                   Q50   0.137575  0.164374  0.190396  0.221498  0.247600   
                   Q95   0.232178  0.273460  0.312556  0.350323  0.389336   
ssp245 Constrained Mean  0.110356  0.133318  0.153856  0.175456  0.196790   
                   Q05   0.057812  0.067891  0.078185  0.089743  0.099656   
                   Q50   0.110490  0.133468  0.154050  0.175708  0.196992   
                   Q95   0.162476  0.198172  0.228897  0.260326  0.293242   
       Raw         Mean  0.142244  0.169002  0.194346  0.221057  0.247328   
                   Q05   0.078538  0.094523  0.110172  0.120932  0.131611   
                   Q50   0.144462  0.169176  0.186691  0.217430  0.246442   
                   Q95   0.236095  0.280563  0.319875  0.355790  0.395013   
ssp370 Constrained Mean  0.110514  0.131077  0.150351  0.170232  0.189356   
                   Q05   0.056332  0.065176  0.076294  0.089199  0.098015   
                   Q50   0.110639  0.131287  0.150591  0.170485  0.189638   
                   Q95   0.164224  0.196265  0.223518  0.250485  0.279686   
       Raw         Mean  0.140902  0.166364  0.190164  0.214140  0.237617   
                   Q05   0.071952  0.088360  0.103678  0.117136  0.125519   
                   Q50   0.140879  0.164493  0.180175  0.203174  0.228663   
                   Q95   0.231550  0.274010  0.315148  0.356425  0.392368   
ssp585 Constrained Mean  0.112166  0.135749  0.158546  0.180372  0.202411   
                   Q05   0.060639  0.074293  0.087185  0.101183  0.114601   
                   Q50   0.112323  0.135931  0.158703  0.180629  0.202637   
                   Q95   0.163359  0.196716  0.229134  0.258803  0.289363   
       Raw         Mean  0.148490  0.176605  0.204208  0.231675  0.259639   
                   Q05   0.082785  0.102707  0.117331  0.145783  0.162305   
                   Q50   0.150533  0.175192  0.200420  0.233042  0.259556   
                   Q95   0.234638  0.279598  0.319145  0.359223  0.404719   

                             2005      2006      2007      2008      2009  \
ssp119 Constrained Mean  0.233025  0.257417  0.284183  0.312529  0.337089   
                   Q05   0.133640  0.148102  0.165138  0.183032  0.194385   
                   Q50   0.233379  0.257861  0.284652  0.312957  0.337514   
                   Q95   0.331167  0.365417  0.401612  0.440416  0.477905   
       Raw         Mean  0.264912  0.293297  0.323926  0.355064  0.382756   
                   Q05   0.175670  0.187608  0.201317  0.222467  0.241877   
                   Q50   0.240301  0.269887  0.300771  0.326335  0.348134   
                   Q95   0.422306  0.474905  0.524845  0.570842  0.617134   
ssp126 Constrained Mean  0.220784  0.245249  0.270528  0.292787  0.316210   
                   Q05   0.118008  0.129419  0.145140  0.157307  0.168071   
                   Q50  

### Ribes et al. (2021)

In [16]:
df_ribes = {}

for scenario in scenarios:
    for label, sfx in [('Constrained', 'cons'), ('Raw', 'uncons')]:
        path = f'input_data/constrained_cmip6/Ribes/ts_hist{scenario}_{sfx}.txt'
        path_local = repo_ch4.retrieve(path)
        df_ribes[(scenario, label)] = pd.read_csv(path_local, sep=' ').T

[2024-07-07 17:02:07 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Ribes/ts_histssp119_cons.txt retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Ribes/ts_histssp119_cons.txt on 2024-06-24
[2024-07-07 17:02:07 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Ribes/ts_histssp119_uncons.txt retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Ribes/ts_histssp119_uncons.txt on 2024-06-24
[2024-07-07 17:02:07 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Ribes/ts_histssp126_cons.txt retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/constrained_cmip6/Ribes/ts_histssp126_cons.txt on 2024-06-24
[2024-07-07 17:02:07 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/constrained_cmip6/Ribes/

In [17]:
df_ribes = pd.concat(df_ribes).rename({'mean': 'Mean'})

In [18]:
slices = [slice(year, year+20-1) for year in df_ribes.columns[:-19]]

In [19]:
df_ribes = pd.concat([
    df_ribes.loc[:, slc].mean(axis=1).to_frame(slc.start)
    for slc in slices
], axis=1)
df_ribes

1850     1851     1852     1853     1854     1855  \
ssp119 Constrained Q05  -0.97120 -0.96905 -0.96705 -0.96515 -0.96360 -0.96140   
                   Mean -0.84660 -0.84565 -0.84475 -0.84390 -0.84335 -0.84210   
                   Q95  -0.72175 -0.72205 -0.72230 -0.72255 -0.72300 -0.72265   
       Raw         Q05  -1.25315 -1.24955 -1.24615 -1.24285 -1.24005 -1.23630   
                   Mean -0.85815 -0.85650 -0.85490 -0.85335 -0.85220 -0.85020   
                   Q95  -0.46300 -0.46325 -0.46355 -0.46380 -0.46435 -0.46405   
ssp126 Constrained Q05  -1.00255 -1.00025 -0.99805 -0.99590 -0.99410 -0.99175   
                   Mean -0.87720 -0.87585 -0.87450 -0.87320 -0.87215 -0.87045   
                   Q95  -0.75180 -0.75135 -0.75085 -0.75035 -0.75005 -0.74905   
       Raw         Q05  -1.27815 -1.27575 -1.27345 -1.27120 -1.26935 -1.26670   
                   Mean -0.88935 -0.88800 -0.88675 -0.88555 -0.88470 -0.88300   
                   Q95  -0.50035 -0.50005 -0.49980 -0.49960 -0.49975 -0.49905   
ssp245 Constrained Q05  -0.99370 -0.99160 -0.98960 -0.98765 -0.98605 -0.98390   
                   Mean -0.86825 -0.86710 -0.86600 -0.86490 -0.86405 -0.86260   
                   Q95  -0.74280 -0.74260 -0.74235 -0.74210 -0.74205 -0.74130   
       Raw         Q05  -1.26775 -1.26550 -1.26330 -1.26115 -1.25940 -1.25695   
                   Mean -0.87120 -0.87005 -0.86895 -0.86790 -0.86720 -0.86575   
                   Q95  -0.47460 -0.47455 -0.47460 -0.47465 -0.47505 -0.47460   
ssp370 Constrained Q05  -0.98535 -0.98345 -0.98160 -0.97985 -0.97840 -0.97640   
                   Mean -0.86100 -0.86000 -0.85905 -0.85810 -0.85740 -0.85610   
                   Q95  -0.73675 -0.73665 -0.73655 -0.73645 -0.73650 -0.73585   
       Raw         Q05  -1.22920 -1.22685 -1.22455 -1.22230 -1.22040 -1.21785   
                   Mean -0.82565 -0.82450 -0.82335 -0.82225 -0.82150 -0.82000   
                   Q95  -0.42205 -0.42205 -0.42205 -0.42215 -0.42250 -0.42210   
ssp585 Constrained Q05  -0.98655 -0.98470 -0.98295 -0.98125 -0.97985 -0.97795   
                   Mean -0.86090 -0.86000 -0.85915 -0.85830 -0.85770 -0.85650   
                   Q95  -0.73550 -0.73555 -0.73555 -0.73550 -0.73565 -0.73515   
       Raw         Q05  -1.23175 -1.22975 -1.22775 -1.22585 -1.22425 -1.22200   
                   Mean -0.83900 -0.83815 -0.83735 -0.83660 -0.83615 -0.83500   
                   Q95  -0.44625 -0.44655 -0.44690 -0.44730 -0.44795 -0.44785   

                            1856     1857     1858     1859  ...     2072  \
ssp119 Constrained Q05  -0.95950 -0.95775 -0.95490 -0.95200  ...  0.23595   
                   Mean -0.84105 -0.84015 -0.83805 -0.83590  ...  0.59345   
                   Q95  -0.72245 -0.72235 -0.72105 -0.71960  ...  0.95065   
       Raw         Q05  -1.23295 -1.22985 -1.22520 -1.22050  ...  0.15790   
                   Mean -0.84850 -0.84695 -0.84390 -0.84075  ...  0.76180   
                   Q95  -0.46395 -0.46390 -0.46250 -0.46095  ...  1.36555   
ssp126 Constrained Q05  -0.98955 -0.98755 -0.98455 -0.98145  ...  0.46910   
                   Mean -0.86890 -0.86750 -0.86500 -0.86240  ...  0.93845   
                   Q95  -0.74815 -0.74735 -0.74540 -0.74330  ...  1.40795   
       Raw         Q05  -1.26435 -1.26220 -1.25875 -1.25520  ...  0.47995   
                   Mean -0.88160 -0.88035 -0.87770 -0.87495  ...  1.19955   
                   Q95  -0.49855 -0.49820 -0.49645 -0.49455  ...  1.91915   
ssp245 Constrained Q05  -0.98200 -0.98020 -0.97745 -0.97465  ...  1.14705   
                   Mean -0.86130 -0.86010 -0.85790 -0.85560  ...  1.69435   
                   Q95  -0.74065 -0.74005 -0.73835 -0.73650  ...  2.24175   
       Raw         Q05  -1.25475 -1.25270 -1.24945 -1.24610  ...  1.15150   
                   Mean -0.86455 -0.86350 -0.86115 -0.85870  ...  1.98790   
                   Q95  -0.47435 -0.47430 -0.47285 -0.47130  ...  2.82440   
ssp370 Constrained Q05  -0.97460 -0.97295 -0.97040 -0

### Combine the three datasets

In [20]:
# In Ribes, treat mean data as median
df_cmip6_combined = pd.concat({
    'liang': df_liang.drop(['Mean'], level=2).loc[:, 2000:],
    'tokarska': df_tokarska.drop(['Mean'], level=2),
    'ribes': df_ribes.loc[:, 2000:].rename({'Mean': 'Q50'}, level=2),
})
df_cmip6_combined

2000      2001      2002      2003  \
liang ssp126 Constrained Q05  0.065606  0.073845  0.084726  0.105236   
                         Q50  0.126918  0.148448  0.170912  0.190793   
                         Q95  0.178574  0.204082  0.236742  0.274865   
             Raw         Q05  0.064892  0.076412  0.089096  0.107756   
                         Q50  0.132784  0.154374  0.176022  0.196888   
...                                ...       ...       ...       ...   
ribes ssp585 Constrained Q50  0.134350  0.158300  0.182100  0.205950   
                         Q95  0.168900  0.196300  0.224000  0.252300   
             Raw         Q05  0.092350  0.114850  0.135900  0.155850   
                         Q50  0.165350  0.195050  0.224500  0.254100   
                         Q95  0.238600  0.275500  0.313400  0.352550   

                                  2004      2005      2006      2007  \
liang ssp126 Constrained Q05  0.122211  0.131038  0.153214  0.171882   
                         Q50  0.214018  0.235476  0.265113  0.293544   
                         Q95  0.308458  0.358036  0.403097  0.440265   
             Raw         Q05  0.122461  0.135308  0.151870  0.170457   
                         Q50  0.227025  0.252821  0.277786  0.309535   
...                                ...       ...       ...       ...   
ribes ssp585 Constrained Q50  0.230400  0.255550  0.281500  0.308200   
                         Q95  0.281700  0.312250  0.344150  0.377300   
             Raw         Q05  0.175450  0.194900  0.214500  0.234450   
                         Q50  0.284400  0.315550  0.347650  0.380650   
                         Q95  0.393550  0.436300  0.480850  0.526950   

                                  2008      2009  ...      2072      2073  \
liang ssp126 Constrained Q05  0.190593  0.213225  ...  0.504059  0.503571   
                         Q50  0.320200  0.343275  ...  0.910395  0.902805   
                         Q95  0.478168  0.529632  ...  1.794165  1.802872   
             Raw         Q05  0.186542  0.211412  ...  0.556459  0.551523   
                         Q50  0.333665  0.363880  ...  1.067305  1.073139   
...                                ...       ...  ...       ...       ...   
ribes ssp585 Constrained Q50  0.335550  0.363450  ...  3.022700  3.077100   
                         Q95  0.411450  0.446350  ...  3.932650  4.006400   
             Raw         Q05  0.254750  0.275400  ...  2.127900  2.163700   
                         Q50  0.414450  0.448900  ...  3.551700  3.613000   
                         Q95  0.574250  0.622500  ...  4.975550  5.062300   

                                  2074      2075      2076      2077  \
liang ssp126 Constrained Q05  0.492218  0.496368  0.498130  0.492889   
                         Q50  0.898204  0.890504  0.885750  0.880515   
                         Q95  1.806384  1.811727  1.814025  1.822304   
             Raw         Q05  0.541493  0.543780  0.544462  0.539050   
                         Q50  1.068402  1.058524  1.056285  1.057785   
...                                ...       ...       ...       ...   
ribes ssp585 Constrained Q50  3.131900  3.186950  3.242250  3.297600   
                         Q95  4.080650  4.155300  4.230250  4.305350   
             Raw         Q05  2.199800  2.236100  2.272450  2.308800   
                         Q50  3.674700  3.736700  3.798900  3.861200   
                         Q95  5.149650  5.237400  5.325450  5.413700   

                                  2078      2079      2080      2081  
liang ssp126 Constrained Q05  0.486969  0.480835  0.474731  0.477739  
                         Q50  0.869330  0.862385  0.866247  0.854275  
                         Q95  1.825708  1.827189  1.829456  1.836880  
             Raw         Q05  0.532467  0.526343  0.519653  0.520152  
                         Q50  1.054159  1.046385  1.036456  1.041266  
...                                ...       ...       ...       ...  
ribes ssp585 Cons

In [21]:
df_cmip6 = df_cmip6_combined.groupby(level=[1, 2, 3]).mean()
df_cmip6

2000      2001      2002      2003      2004  \
ssp119 Constrained Q05  0.081655  0.099411  0.114441  0.127430  0.140167   
                   Q50  0.121787  0.144306  0.164532  0.185124  0.205591   
                   Q95  0.161433  0.188683  0.214028  0.242049  0.270087   
       Raw         Q05  0.076427  0.096977  0.114766  0.130586  0.145778   
                   Q50  0.142760  0.164216  0.183659  0.209895  0.233693   
                   Q95  0.227801  0.264267  0.299520  0.334794  0.370122   
ssp126 Constrained Q05  0.071135  0.083597  0.097313  0.114405  0.129510   
                   Q50  0.120487  0.142183  0.163661  0.184358  0.206197   
                   Q95  0.166399  0.194164  0.223032  0.253469  0.283413   
       Raw         Q05  0.078539  0.095072  0.109496  0.127950  0.143845   
                   Q50  0.142603  0.167366  0.191623  0.217179  0.244242   
                   Q95  0.225400  0.260901  0.297496  0.334201  0.371553   
ssp245 Constrained Q05  0.072510  0.086645  0.099119  0.114874  0.126411   
                   Q50  0.121567  0.145859  0.167304  0.189395  0.212210   
                   Q95  0.170848  0.196171  0.223017  0.253302  0.284620   
       Raw         Q05  0.076839  0.094157  0.108358  0.121569  0.134183   
                   Q50  0.146028  0.171641  0.194412  0.222845  0.249491   
                   Q95  0.223824  0.261808  0.300419  0.337781  0.375922   
ssp370 Constrained Q05  0.072633  0.084559  0.096945  0.110678  0.122425   
                   Q50  0.119620  0.142870  0.164239  0.184514  0.205048   
                   Q95  0.176758  0.207302  0.238197  0.261057  0.288794   
       Raw         Q05  0.072061  0.087178  0.101208  0.115183  0.126858   
                   Q50  0.141890  0.165391  0.186948  0.210910  0.236686   
                   Q95  0.225988  0.263438  0.301101  0.339135  0.374909   
ssp585 Constrained Q05  0.075995  0.090413  0.105652  0.120179  0.137177   
                   Q50  0.124532  0.148628  0.171520  0.194900  0.217331   
                   Q95  0.177357  0.210061  0.245204  0.276672  0.308451   
       Raw         Q05  0.080238  0.098207  0.113992  0.134712  0.151504   
                   Q50  0.151139  0.177492  0.203257  0.232660  0.259837   
                   Q95  0.229337  0.269844  0.310663  0.349269  0.391356   

                            2005      2006      2007      2008      2009  ...  \
ssp119 Constrained Q05  0.149895  0.164176  0.179719  0.195641  0.208142  ...   
                   Q50  0.225290  0.247106  0.270201  0.294129  0.316132  ...   
                   Q95  0.299658  0.328859  0.359331  0.391308  0.422627  ...   
       Raw         Q05  0.157160  0.168754  0.181208  0.197384  0.212589  ...   
                   Q50  0.253276  0.279743  0.307036  0.331743  0.354492  ...   
                   Q95  0.408053  0.452077  0.495123  0.536321  0.577642  ...   
ssp126 Constrained Q05  0.141099  0.157411  0.173974  0.189333  0.205482  ...   
                   Q50  0.228077  0.253067  0.277988  0.301386  0.324042  ...   
                   Q95  0.320474  0.356674  0.389672  0.422218  0.460491  ...   
       Raw         Q05  0.158608  0.173088  0.189403  0.205931  0.225291  ...   
                   Q50  0.265955  0.293559  0.324000  0.349770  0.378044  ...   
                   Q95  0.411096  0.452165  0.496554  0.540817  0.584576  ...   
ssp245 Constrained Q05  0.142465  0.161474  0.177835  0.191570  0.206077  ...   
                   Q50  0.235542  0.256647  0.279456  0.302361  0.325894  ...   
                   Q95  0.316494  0.349761  0.383370  0.418140  0.454246  ...   
       Raw         Q05  0.153746  0.175033  0.195284  0.209522  0.226778  ...   
                   Q50  0.273759  0.296452  0.321425  0.349169  0.379817  ...   
                   Q95  0.418660  0.464449  0.508915  0.553381  0.599120  ...   
ssp370 Constrained Q05  0.135522  0.148593  0.165287  0.181503  0.196423  ...   
                   Q50  0.225119  0.247154  0.27191

## Emulator projections

### ERF time series from the chapter 7 assessment

In [22]:
df_erf = {}

for scenario in scenarios:
    path = f'input_data/ssp_erf/ERF_{scenario}_1750-2500.csv'
    path_local = repo_ch4.retrieve(path)
    df_erf[scenario] = pd.read_csv(path_local, index_col=0)

[2024-07-07 17:02:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/ssp_erf/ERF_ssp119_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/ssp_erf/ERF_ssp119_1750-2500.csv on 2024-06-24
[2024-07-07 17:02:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/ssp_erf/ERF_ssp126_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/ssp_erf/ERF_ssp126_1750-2500.csv on 2024-06-24
[2024-07-07 17:02:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/ssp_erf/ERF_ssp245_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_data/ssp_erf/ERF_ssp245_1750-2500.csv on 2024-06-24
[2024-07-07 17:02:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/input_data/ssp_erf/ERF_ssp370_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/input_

In [23]:
df_erf = pd.concat(df_erf, names=['scenario'])

In [24]:
# Adjust total and CO2 forcing relative to 1850, and volcanic forcing is separately added
df = (df_erf['total'] - df_erf['volcanic']).unstack('year')
df_erf_total = df.sub(df[1850], axis=0).add(df_erf['volcanic'].unstack('year'))

df = df_erf['co2'].unstack('year')
df_erf_co2 = df.sub(df[1850], axis=0)

### Run the emulator with Chapter 4 configuration

In [25]:
def twolayermodel(forcing_years, input_forcing, f2x=4.0, ECS=3.0, TCR=1.8, T_ml0=0.0, T_deep0=0.0):
    """
    inputs:
    forcing_years: a list of the years in the input_forcing
    input_forcing: the yearly forcing value
    ECS:           the equilibrium climate sensitivity
    TCR:           the transient climate response
    T_ml0:         the mixed-layer initial temperature
    T_deep0:       the deep ocean initial temperature

    alpha:         climate feedback parameter, computed from ERF and ECS
    kappa:         ocean heat uptake efficiency, computed from ERF, ECS, and TCR
    """
    # settings for emulator
    # Define time parameters in seconds:
    hour       = 3600
    day        = 24*hour
    month      = 30*day
    year       = 12*month

    nyears     = np.size(forcing_years)
    delta_time = month
    nstep      = round(nyears*year/delta_time)
    time       = delta_time*np.arange(0,nstep,dtype=float) + min(forcing_years)*year
    timeyears  = time/year
    forcing    = np.interp(timeyears,forcing_years,input_forcing)

    # Parameters:
    density    = 1000.
    c_w        = 4181.
    C_ml       = 50*density*c_w # assuming 50m mixed layer
    C_deep     = 1200*density*c_w # assuming 1200m depp ocean
    alpha      = f2x/ECS
    
    kappa = f2x/TCR - f2x/ECS
    
    # Initialize state variables:
    T_ml       = np.zeros(nstep)
    T_deep     = np.zeros(nstep)
    T_ml[0]    = T_ml0
    T_deep[0]  = T_deep0
    imbalance  = np.zeros(nstep)
    
    # Integrate:
    imbalance[0] = np.nan 
    for t in range(0, nstep-1):
        T_ml[t+1]      = (
            T_ml[t] + ( forcing[t] - (alpha*T_ml[t]) - (kappa*(T_ml[t]-T_deep[t])) ) * delta_time/C_ml
        )
        T_deep[t+1]    = T_deep[t] + ( kappa*(T_ml[t]-T_deep[t]) * delta_time/C_deep )
        # this imbalance should be changed to include the efficacy term
        # if kappa is gamma times epsilon,
        # where gamma is the heat exchange coefficient, and epsilon is the efficacy factor
        imbalance[t+1] = (C_ml*(T_ml[t+1]-T_ml[t]) + C_deep*(T_deep[t+1]-T_deep[t]))/delta_time
    
    # Output result and settings for this run:
    result = {
        'time': timeyears,
        'forcing': forcing,
        'T_ml': T_ml,
        'T_deep': T_deep,
        'imbalance': imbalance,
        'ECS': ECS,
        'TCR': TCR,
        'C_ml': C_ml,
        'C_deep': C_deep,
        'kappa': kappa,
        'alpha': alpha,
    }
    return result

In [26]:
# Forcing from a doubling of CO2:
f2x        = 4.0

# central estimates for ECS and TCR
ECS = 3
TCR = 1.8

# very likely and likely range for ECS
ECS_vll = 2 # very likely low
ECS_ll  = 2.5 # likely low
ECS_lh  = 4 # likely high
ECS_vlh = 5 # very likely high

# very likely and likely range for TCR
TCR_vll = 1.2 # very likely low
TCR_ll  = 1.4 # likely low
TCR_lh  = 2.2 # likely high
TCR_vlh = 2.4 # very likely high

In [27]:
vnames = ['time', 'forcing', 'T_ml', 'T_deep', 'imbalance']
pnames = ['ECS', 'TCR', 'C_ml', 'C_deep', 'kappa', 'alpha']

results = {}
parms = {}

for scenario in scenarios:
    d1 = df_erf_total.loc[scenario, 1850:]

    for ecs, tcr in zip(
        [ECS, ECS_vll, ECS_ll, ECS_lh, ECS_vlh],
        [TCR, TCR_vll, TCR_ll, TCR_lh, TCR_vlh],
    ):
        result = twolayermodel(d1.index.values, d1.values, ECS=ecs, TCR=tcr)
        k = tuple([scenario] + [result[pn] for pn in pnames[:2]])
        results[k] = pd.DataFrame({
            variable: result[variable]
            for variable in vnames
        }).set_index('time').T.rename_axis('variable')
        parms[k] = pd.Series({
            pn: result[pn] for pn in pnames[2:]
        }).rename_axis('parameter')

In [28]:
df_results = pd.concat(results, names=['scenario']+pnames[:2])
df_results

time                        1850.000000  1850.083333  1850.166667  \
scenario ECS TCR variable                                           
ssp119   3.0 1.8 forcing       0.194333     0.195180     0.196028   
                 T_ml          0.000000     0.002410     0.004763   
                 T_deep        0.000000     0.000000     0.000001   
                 imbalance          NaN     0.194333     0.191968   
         2.0 1.2 forcing       0.194333     0.195180     0.196028   
...                                 ...          ...          ...   
ssp585   4.0 2.2 imbalance          NaN     0.194333     0.192771   
         5.0 2.4 forcing       0.194333     0.195180     0.196028   
                 T_ml          0.000000     0.002410     0.004780   
                 T_deep        0.000000     0.000000     0.000001   
                 imbalance          NaN     0.194333     0.193253   

time                        1850.250000  1850.333333  1850.416667  \
scenario ECS TCR variable                                           
ssp119   3.0 1.8 forcing       0.196876     0.197724     0.198572   
                 T_ml          0.007062     0.009309     0.011504   
                 T_deep        0.000003     0.000007     0.000011   
                 imbalance     0.189677     0.187459     0.185312   
         2.0 1.2 forcing       0.196876     0.197724     0.198572   
...                                 ...          ...          ...   
ssp585   4.0 2.2 imbalance     0.191253     0.189778     0.188345   
         5.0 2.4 forcing       0.196876     0.197724     0.198572   
                 T_ml          0.007112     0.009406     0.011663   
                 T_deep        0.000003     0.000006     0.000011   
                 imbalance     0.192204     0.191187     0.190199   

time                        1850.500000  1850.583333  1850.666667  \
scenario ECS TCR variable                                           
ssp119   3.0 1.8 forcing       0.199419     0.200267     0.201115   
                 T_ml          0.013649     0.015746     0.017796   
                 T_deep        0.000016     0.000022     0.000030   
                 imbalance     0.183233     0.181220     0.179272   
         2.0 1.2 forcing       0.199419     0.200267     0.201115   
...                                 ...          ...          ...   
ssp585   4.0 2.2 imbalance     0.186952     0.185600     0.184286   
         5.0 2.4 forcing       0.199419     0.200267     0.201115   
                 T_ml          0.013884     0.016070     0.018221   
                 T_deep        0.000016     0.000022     0.000029   
                 imbalance     0.189241     0.188312     0.187411   

time                        1850.750000  ...  2500.166667  2500.250000  \
scenario ECS TCR variable                ...                             
ssp119   3.0 1.8 forcing       0.201963  ...     1.305637     1.305637   
                 T_ml          0.019799  ...     0.965336     0.965327   
                 T_deep        0.000038  ...     0.943734     0.943744   
                 imbalance     0.177388  ...     0.018511     0.018522   
         2.0 1.2 forcing       0.201963  ...     1.305637     1.305637   
...                                 ...  ...          ...          ...   
ssp585   4.0 2.2 imbalance     0.183011  ...     1.527680     1.527414   
         5.0 2.4 forcing       0.201963  ...    12.133878    12.133878   
                 T_ml          0.020339  ...    12.718246    12.718673   
                 T_deep        0.000037  ...    10.497323    10.498317   
                 imbalance     0.186538  ...     1.959622     1.959281   

time                        2500.333333  2500.416667  2500.500000  \
scenario ECS TCR variable                                           
ssp119   3.0 1.8 forcing       1.305637     1.305637     1.305637   
                 T_ml          0.965319     0.965312     0.965304   
                 T_deep        0.943754     0.943764     0.943774  

In [29]:
df_parms = pd.concat(parms, names=['scenario']+pnames[:2]).unstack('parameter')
df_parms

parameter                C_ml        C_deep     kappa     alpha
scenario ECS TCR                                               
ssp119   2.0 1.2  209050000.0  5.017200e+09  1.333333  2.000000
         2.5 1.4  209050000.0  5.017200e+09  1.257143  1.600000
         3.0 1.8  209050000.0  5.017200e+09  0.888889  1.333333
         4.0 2.2  209050000.0  5.017200e+09  0.818182  1.000000
         5.0 2.4  209050000.0  5.017200e+09  0.866667  0.800000
ssp126   2.0 1.2  209050000.0  5.017200e+09  1.333333  2.000000
         2.5 1.4  209050000.0  5.017200e+09  1.257143  1.600000
         3.0 1.8  209050000.0  5.017200e+09  0.888889  1.333333
         4.0 2.2  209050000.0  5.017200e+09  0.818182  1.000000
         5.0 2.4  209050000.0  5.017200e+09  0.866667  0.800000
ssp245   2.0 1.2  209050000.0  5.017200e+09  1.333333  2.000000
         2.5 1.4  209050000.0  5.017200e+09  1.257143  1.600000
         3.0 1.8  209050000.0  5.017200e+09  0.888889  1.333333
         4.0 2.2  209050000.0  5.017200e+09  0.818182  1.000000
         5.0 2.4  209050000.0  5.017200e+09  0.866667  0.800000
ssp370   2.0 1.2  209050000.0  5.017200e+09  1.333333  2.000000
         2.5 1.4  209050000.0  5.017200e+09  1.257143  1.600000
         3.0 1.8  209050000.0  5.017200e+09  0.888889  1.333333
         4.0 2.2  209050000.0  5.017200e+09  0.818182  1.000000
         5.0 2.4  209050000.0  5.017200e+09  0.866667  0.800000
ssp585   2.0 1.2  209050000.0  5.017200e+09  1.333333  2.000000
         2.5 1.4  209050000.0  5.017200e+09  1.257143  1.600000
         3.0 1.8  209050000.0  5.017200e+09  0.888889  1.333333
         4.0 2.2  209050000.0  5.017200e+09  0.818182  1.000000
         5.0 2.4  209050000.0  5.017200e+09  0.866667  0.800000

In [30]:
df_gsat = df_results.groupby('variable').get_group('T_ml').droplevel('variable')
bl = (df_gsat.columns >= 1995) & (df_gsat.columns < 2014+1) 
df_gsat = df_gsat.sub(df_gsat.loc[:, bl].mean(axis=1), axis=0)

In [31]:
df_gsat_ann = df_gsat.apply(
    lambda d1:
    pd.Series(
        d1.values.reshape((-1, 12)).mean(axis=1),
        index=d1.index[::12],
    ),
    axis=1,
)

In [32]:
map_quantile = {
    (ECS_vll, TCR_vll): 'Q05',
    (ECS_ll, TCR_ll): 'Q17',
    (ECS, TCR): 'Q50',
    (ECS_lh, TCR_lh): 'Q83',
    (ECS_vlh, TCR_vlh): 'Q95',
}

In [33]:
df = df_gsat_ann.rename(columns=int)
df.index = pd.MultiIndex.from_tuples([
    (x[0], map_quantile[x[1:]]) for x in df.index
])
df = df.sort_index()

In [34]:
years = pd.Index(range(1850, 2081+1))

df = pd.concat([
    df.loc[:, year:year+19].mean(axis=1)
    for year in years
], axis=1)

df.columns = years

In [35]:
df_emulator = df

## Post processing

In [36]:
df_combined = pd.concat({
    'CMIP6 constrained':
    df_cmip6.loc[(slice(None), 'Constrained', slice(None))],
    'Emulator':
    df_emulator.drop(['Q17', 'Q83'], level=1).loc[:, 2000:],
})
df_combined

2000      2001      2002      2003  \
CMIP6 constrained ssp119 Q05  0.081655  0.099411  0.114441  0.127430   
                         Q50  0.121787  0.144306  0.164532  0.185124   
                         Q95  0.161433  0.188683  0.214028  0.242049   
                  ssp126 Q05  0.071135  0.083597  0.097313  0.114405   
                         Q50  0.120487  0.142183  0.163661  0.184358   
                         Q95  0.166399  0.194164  0.223032  0.253469   
                  ssp245 Q05  0.072510  0.086645  0.099119  0.114874   
                         Q50  0.121567  0.145859  0.167304  0.189395   
                         Q95  0.170848  0.196171  0.223017  0.253302   
                  ssp370 Q05  0.072633  0.084559  0.096945  0.110678   
                         Q50  0.119620  0.142870  0.164239  0.184514   
                         Q95  0.176758  0.207302  0.238197  0.261057   
                  ssp585 Q05  0.075995  0.090413  0.105652  0.120179   
                         Q50  0.124532  0.148628  0.171520  0.194900   
                         Q95  0.177357  0.210061  0.245204  0.276672   
Emulator          ssp119 Q05  0.098589  0.114335  0.129807  0.145833   
                         Q50  0.149683  0.174144  0.197809  0.221673   
                         Q95  0.203722  0.238367  0.271867  0.305274   
                  ssp126 Q05  0.098478  0.114104  0.129375  0.145106   
                         Q50  0.149558  0.173879  0.197308  0.220816   
                         Q95  0.203589  0.238083  0.271323  0.304336   
                  ssp245 Q05  0.098562  0.114319  0.129720  0.145528   
                         Q50  0.149648  0.174120  0.197711  0.221335   
                         Q95  0.203682  0.238337  0.271761  0.304915   
                  ssp370 Q05  0.097528  0.112463  0.126880  0.141669   
                         Q50  0.148470  0.171960  0.194326  0.216614   
                         Q95  0.202419  0.235991  0.268028  0.299627   
                  ssp585 Q05  0.099436  0.116012  0.132541  0.149768   
                         Q50  0.150636  0.176070  0.201030  0.226423   
                         Q95  0.204738  0.240445  0.275392  0.310552   

                                  2004      2005      2006      2007  \
CMIP6 constrained ssp119 Q05  0.140167  0.149895  0.164176  0.179719   
                         Q50  0.205591  0.225290  0.247106  0.270201   
                         Q95  0.270087  0.299658  0.328859  0.359331   
                  ssp126 Q05  0.129510  0.141099  0.157411  0.173974   
                         Q50  0.206197  0.228077  0.253067  0.277988   
                         Q95  0.283413  0.320474  0.356674  0.389672   
                  ssp245 Q05  0.126411  0.142465  0.161474  0.177835   
                         Q50  0.212210  0.235542  0.256647  0.279456   
                         Q95  0.284620  0.316494  0.349761  0.383370   
                  ssp370 Q05  0.122425  0.135522  0.148593  0.165287   
                         Q50  0.205048  0.225119  0.247154  0.271910   
                         Q95  0.288794  0.318116  0.349195  0.382017   
                  ssp585 Q05  0.137177  0.153969  0.172433  0.191251   
                         Q50  0.217331  0.239365  0.263518  0.291312   
                         Q95  0.308451  0.344655  0.378702  0.413825   
Emulator          ssp119 Q05  0.162933  0.181034  0.200152  0.220126   
                         Q50  0.246474  0.272324  0.299362  0.327510   
                         Q95  0.339442  0.374599  0.410985  0.448605   
                  ssp126 Q05  0.161824  0.179473  0.198096  0.217563   
                         Q50  0.245144  0.270418  0.296806  0.324266   
                         Q95  0.337968  0.372464  0.408087  0.444881   
                  ssp245 Q05  0.162249  0.179824  0.198301  0.217561   
                         Q50  0.245698  0.270917  0.297156  0.324382   
                         Q95  0.338610  0.373070  0.408553  0.445106   
 

In [37]:
df_assessed = {}

for scenario in scenarios:
    path = f'assessed_GSAT_projections/assessed_GSAT_{scenario}.nc'
    path_local = repo_ch4.retrieve(path)
    ncf = Dataset(path_local)
    df = pd.DataFrame({
        name: ncf.variables[name][:]
        for name in ncf.variables
    })
    units = ncf.variables['time'].units
    calendar = ncf.variables['time'].calendar
    df.index = [t1.year for t1 in num2date(df['time'], units, calendar)]
    ncf.close()
    df_assessed[scenario] = df.drop('time', axis=1).T

df_assessed = pd.concat(df_assessed)

[2024-07-07 17:03:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/assessed_GSAT_projections/assessed_GSAT_ssp119.nc retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/assessed_GSAT_projections/assessed_GSAT_ssp119.nc on 2024-06-24
[2024-07-07 17:03:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/assessed_GSAT_projections/assessed_GSAT_ssp126.nc retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/assessed_GSAT_projections/assessed_GSAT_ssp126.nc on 2024-06-24
[2024-07-07 17:03:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/assessed_GSAT_projections/assessed_GSAT_ssp245.nc retrieved from https://github.com/IPCC-WG1/Chapter-4_Figure4.11/raw/main/assessed_GSAT_projections/assessed_GSAT_ssp245.nc on 2024-06-24
[2024-07-07 17:03:30 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-4_Figure4.11/assessed_GSAT_projections/assessed_GSAT_ssp370.nc retrieved from https://github.com/IPC

In [38]:
df_assessed

2000      2001      2002      2003      2004      2005  \
ssp119 Q05  0.090122  0.106873  0.122124  0.136631  0.151550  0.165465   
       Q50  0.135735  0.159225  0.181170  0.203398  0.226032  0.248807   
       Q95  0.182578  0.213525  0.242947  0.273661  0.304764  0.337129   
ssp126 Q05  0.084806  0.098850  0.113344  0.129756  0.145667  0.160286   
       Q50  0.135023  0.158031  0.180485  0.202587  0.225670  0.249248   
       Q95  0.184994  0.216124  0.247178  0.278902  0.310690  0.346469   
ssp245 Q05  0.085536  0.100482  0.114419  0.130201  0.144330  0.161145   
       Q50  0.135607  0.159989  0.182507  0.205365  0.228954  0.253229   
       Q95  0.187265  0.217254  0.247389  0.279108  0.311615  0.344782   
ssp370 Q05  0.085081  0.098511  0.111913  0.126173  0.139921  0.154820   
       Q50  0.134045  0.157415  0.179282  0.200564  0.222343  0.244355   
       Q95  0.189588  0.221646  0.253113  0.280342  0.310253  0.341357   
ssp585 Q05  0.087715  0.103213  0.119097  0.134973  0.152689  0.170885   
       Q50  0.137584  0.162349  0.186275  0.210661  0.235150  0.260088   
       Q95  0.191047  0.225253  0.260298  0.293612  0.327605  0.364465   

                2006      2007      2008      2009  ...      2072      2073  \
ssp119 Q05  0.182164  0.199922  0.218048  0.234379  ...  0.279081  0.275923   
       Q50  0.273234  0.298856  0.325205  0.350604  ...  0.605213  0.601043   
       Q95  0.369922  0.403968  0.439161  0.474116  ...  0.997349  0.992423   
ssp126 Q05  0.177754  0.195768  0.213368  0.231301  ...  0.541900  0.538848   
       Q50  0.274936  0.301127  0.326867  0.352266  ...  0.933812  0.930553   
       Q95  0.382381  0.417277  0.452330  0.490351  ...  1.477555  1.478884   
ssp245 Q05  0.179888  0.197698  0.214357  0.231322  ...  1.167824  1.175909   
       Q50  0.276902  0.301919  0.327257  0.352911  ...  1.692411  1.707050   
       Q95  0.379157  0.414238  0.450236  0.486965  ...  2.400692  2.422186   
ssp370 Q05  0.170226  0.187919  0.205631  0.222718  ...  1.733028  1.761603   
       Q50  0.267929  0.293451  0.319831  0.344816  ...  2.404365  2.443177   
       Q95  0.373911  0.407962  0.443766  0.483081  ...  3.253913  3.307411   
ssp585 Q05  0.190535  0.210933  0.229272  0.249179  ...  2.122805  2.157826   
       Q50  0.286830  0.316139  0.345275  0.373858  ...  3.035569  3.089076   
       Q95  0.401050  0.439023  0.479559  0.523857  ...  4.150795  4.223204   

                2074      2075      2076      2077      2078      2079  \
ssp119 Q05  0.272325  0.270064  0.265474  0.259808  0.256181  0.251440   
       Q50  0.596573  0.593611  0.589095  0.584458  0.579552  0.573898   
       Q95  0.987981  0.984985  0.981303  0.977549  0.972303  0.966792   
ssp126 Q05  0.533839  0.531854  0.528969  0.524163  0.518899  0.514902   
       Q50  0.927842  0.924577  0.922189  0.919014  0.914498  0.910810   
       Q95  1.479567  1.480611  1.481367  1.482966  1.482888  1.482466   
ssp245 Q05  1.183985  1.194171  1.204353  1.212798  1.220566  1.228155   
       Q50  1.721728  1.736203  1.750524  1.764443  1.777414  1.788887   
       Q95  2.442892  2.462704  2.484217  2.505945  2.526209  2.546525   
ssp370 Q05  1.790273  1.818919  1.846718  1.877168  1.910145  1.939998   
       Q50  2.482337  2.521379  2.559119  2.598358  2.639697  2.679517   
       Q95  3.363172  3.418736  3.473225  3.527564  3.582394  3.634234   
ssp585 Q05  2.193164  2.228832  2.265182  2.301822  2.335778  2.372940   
       Q50  3.140479  3.192170  3.243119  3.295186  3.347856  3.399290   
       Q95  4.297715  4.371882  4.446121  4.521190  4.596819  4.670999   

                2080      2081  
ssp119 Q05  0.245231  0.242254  
       Q50  0.567881  0.564310  
       Q95  0.961103  0.957548  
ssp126 Q05  0.510257  0.506951  
       Q50  0.908730  0.904316  
       Q95  1.481471  1.482649  
ssp245 Q05  1.235346  1.243935  
       Q50  1.800092  1.814827  
       Q95  2.565727  2.585020  
ssp370 Q05  1.970434  1.997465  
       Q50  2.7176

In [39]:
np.allclose(
    df_assessed,
    df_combined.groupby(level=[1, 2]).mean(),
)

True

In [40]:
df_cmip6.to_csv('./dataout/ch4_cmip6.csv')
df_emulator.to_csv('./dataout/ch4_emulator.csv')